In [25]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [269]:
P = pd.read_csv("cap_cal_data.csv")
P.head()

,Unnamed: 0,ADEP,ADES,FiledOBT,FiledAT,ACType,ACOperator,ArrivalDelay,DepartureDelay,month,weekday,filedATminutes,filedOBTminutes
0,420,UUDD,EGLL,2019-03-01 02:40:00,2019-03-01 06:15:15,A321,BAW,23.0,12.0,3,4,375,160
1,1173,UUEE,EGLL,2019-03-01 05:00:00,2019-03-01 08:36:02,A321,AFL,25.0,10.0,3,4,516,300
2,1661,UUEE,EGLL,2019-03-01 05:25:00,2019-03-01 08:59:34,B738,AFL,27.0,11.0,3,4,539,325
3,1687,LFLL,EGLL,2019-03-01 05:29:00,2019-03-01 06:50:53,A320,BAW,17.0,7.0,3,4,410,329
4,1945,LOWW,EGLL,2019-03-01 05:40:00,2019-03-01 07:43:12,A321,AUA,8.0,6.0,3,4,463,340


In [272]:
def capacity_calc(P: pd.DataFrame, airport_capacity: int=80):
    dform = "%Y-%m-%d %H:%M:%S"
    P = P.assign(FiledOBT=lambda x: pd.to_datetime(x.FiledOBT, format=dform)).assign(FiledAT=lambda x: pd.to_datetime(x.FiledAT, format=dform))
    airport = "EGLL"
    # times = pd.DatetimeIndex(P.time)

    dep = P.query("ADEP == @airport")
    des = P.query("ADES == @airport")
    dep = dep.assign(Date = lambda x: x.FiledOBT.dt.date)
    des = des.assign(Date = lambda x: x.FiledAT.dt.date)

    dep = dep.assign(Hour = lambda x: x.FiledOBT.dt.hour).assign(Minutes= lambda x: x.FiledOBT.dt.minute //15 * 15).assign(Time= lambda x: x.FiledOBT)
    des = des.assign(Hour = lambda x: x.FiledAT.dt.hour).assign(Minutes= lambda x: x.FiledAT.dt.minute //15 * 15).assign(Time= lambda x: x.FiledAT)

    new_df = pd.concat([dep,des], axis=0)
    new_df.Time = new_df.Time.apply(lambda x: pd.datetime(x.year, x.month, x.day, x.hour, x.minute//15 * 15, 0))
    times = pd.DatetimeIndex(new_df.Time)
    K = new_df.groupby([times.date, times.hour, times.minute])["ADES"].count()
    cap_dict = K.to_dict()

    new_df["Time_tuple"] = list(zip(new_df.Date, new_df.Hour, new_df.Minutes))
    new_df["capacity"] = new_df["Time_tuple"].map(cap_dict) / airport_capacity
    new_df = new_df.drop(["Time_tuple", "Date", "Hour", "Minutes", "Time"], axis=1)
    new_df = new_df.sort_values("FiledAT")
    return new_df

df = capacity_calc(P, 88)
df.head()

C:\Users\niels\AppData\Local\Temp/ipykernel_15788/348295654.py:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  new_df.Time = new_df.Time.apply(lambda x: pd.datetime(x.year, x.month, x.day, x.hour, x.minute//15 * 15, 0))


,Unnamed: 0,ADEP,ADES,FiledOBT,FiledAT,ACType,ACOperator,ArrivalDelay,DepartureDelay,month,weekday,filedATminutes,filedOBTminutes,capacity
0,420,UUDD,EGLL,2019-03-01 02:40:00,2019-03-01 06:15:15,A321,BAW,23.0,12.0,3,4,375,160,0.045455
6,2232,EDDL,EGLL,2019-03-01 05:50:00,2019-03-01 06:50:46,A320,EWG,19.0,4.0,3,4,410,350,0.102273
3,1687,LFLL,EGLL,2019-03-01 05:29:00,2019-03-01 06:50:53,A320,BAW,17.0,7.0,3,4,410,329,0.102273
29,2975,EGCC,EGLL,2019-03-01 06:19:00,2019-03-01 07:05:29,A319,SHT,10.0,8.0,3,4,425,379,0.170455
32,2988,EHAM,EGLL,2019-03-01 06:20:00,2019-03-01 07:07:31,B744,SIA,33.0,-2.0,3,4,427,380,0.170455


30.0
